# Dynamic Load Balancing with Tokens

Compute the performance of two static load balancing policies, referred to as *best static* and *uniform static*.

## Package imports and global variable definitions

In [ ]:
%pylab inline

In [ ]:
# uncomment this line if you prefer dynamic matplotlib plots
# %matplotlib notebook

# change the default figure size
pylab.rcParams['figure.figsize'] = (10.0, 6.0)
pylab.rcParams['legend.fontsize'] = 12

In [ ]:
# manipulate dataframes
import pandas as pd

In [ ]:
# global variables
ρρ = append( linspace(5., .01, 400, endpoint = False), linspace(.01, 0, 10, endpoint = False) )
nb_tokens = 6

## Performance of the static load balancing policies

### Computations

** Independence.**
In steady state, the number of jobs at each server is independent of the number of jobs at the other servers.
Therefore, the stationary distribution of the aggregate state $y = (y_i : i = 1, \ldots, N)$
that counts the number of available tokens of each server
is given by
$$
\pi(y) = \pi(0) \prod_{i=1}^N {\rho_i}^{y_i},
\quad \forall y \le \ell,
$$
where $\rho_1 = \frac{\mu_1}{\nu p_1}, \ldots, \rho_N = \frac{\mu_N}{\nu p_N}$
are the (state-independent) loads at the servers.

** Blocking probability.**
With this observation in mind, we can write the average blocking probability $\beta$ as follows:
$$
\beta
= \sum_{i=1}^N p_i \beta_{|i},
$$
where $\beta_{|i}$ is the blocking probability of jobs assigned to server $i$.
The arrival process of jobs assigned to server $i$ is Poisson with rate $\lambda_i = \nu p_i$.
Hence, we can apply PASTA property, which states that
$\beta_{|i}$ is the probability that all tokens of server $i$ are held by jobs in service,
that is, $y_i = 0$:
$$
\beta_{|i} = \frac1{ \sum_{y_i = 0}^{\ell_i} {\rho_i}^{y_i} }.
$$

**Mean number of jobs.**
By linearity of the expectation, the mean number $L$ of jobs in the system is given by
$$
L = \sum_{i=1}^N L_i,
$$
where, for each $i = 1,\ldots,N$,
$L_i$ is the mean number of jobs at server $i$.
Since the servers states are independent, we have
$$
L_i = \frac
{ \sum_{y_i = 0}^{\ell_i} y_i {\rho_i}^{y_i} }
{ \sum_{y_i = 0}^{\ell_i} {\rho_i}^{y_i} },
\quad \forall i = 1,\ldots,N.
$$

### Functions

In [ ]:
def mm1ℓ(ℓ, ρ):
    π = power(ρ, arange(ℓ+1))
    π /= sum(π)
    return π[0], ℓ - inner(π, arange(ℓ+1))

In [ ]:
def static(N, ℓ, μ, ν, p):
    β = zeros(N); L = zeros(N)
    result = []
    
    for i in range(N):
        β[i], L[i] = mm1ℓ(ℓ[i], μ[i] / (ν * p[i]))
        result += (β[i], L[i])
    result += [inner(p, β), sum(L)]
    
    return result

The function ``create_static_df`` calls the function ``static`` and creates a dataframe (from ``pandas`` library) that stores the results:

In [ ]:
def create_static_df(N, ℓ, μ, ρρ, p, prefix = ""):
    data = [static(N, ℓ, μ, ρ * sum(μ), p) for ρ in ρρ]
    
    df = pd.DataFrame({'rho': ρρ})
    
    for i in range(N):
        df[prefix + 'betai' + str(i+1)] = [d[2*i] for d in data]
        df[prefix + 'psii' + str(i+1)] = 1. - (sum(μ) * p[i] / μ[i]) * ρρ * [1. - d[2*i] for d in data]
        df[prefix + 'Li' + str(i+1)] = [d[2*i+1] for d in data]
        df[prefix + 'gammai' + str(i+1)] = ρρ * sum(μ) * p[i] * [(1. - d[2*i]) / d[2*i+1] for d in data]

    df[prefix + 'beta'] = [d[2*N] for d in data]
    df[prefix + 'eta'] = ρρ * [1. - d[2*N] for d in data]
    df[prefix + 'L'] = [d[2*N+1] for d in data]
    df[prefix + 'gamma'] = ρρ * sum(μ) * [(1. - d[2*N]) / d[2*N+1] for d in data]
    
    return df

## A single job type

We consider the first scenario in the paper.
There are $N = 10$ servers, each with $\ell = 6$ tokens.
The first half have a unit service capacity $\mu$
and the other half have a service capacity $4 \mu$.
There is a single job type, i.e., all jobs can be assigned to any server.
The external arrival rate is denoted by $\nu$.

In [ ]:
# parameters
N = 10
μ = ones(N); μ[N//2:] = 4.
ℓ = nb_tokens * ones(N, dtype = int)

# best static policy
best = ones(N)
best[N//2:] = 4.
best /= sum(best)

# uniform static policy
uni = ones(N)
uni /= sum(uni)

In [ ]:
# compute the analytical results
best_static_df = create_static_df(N, ℓ, μ, ρρ, best)
uni_static_df = create_static_df(N, ℓ, μ, ρρ, uni)

In [ ]:
# save them in csv
best_static_df.to_csv("data/single-best-static-exact.csv", index = False)
uni_static_df.to_csv("data/single-uni-static-exact.csv", index = False)

## Two job types

We consider the second scenario in the paper.
There are $N = 10$ servers, each with $\ell = 6$ tokens.
All servers have the same unit service capacity $\mu$.
There are two job types.
The jobs of the first type arrive at a unit rate $\nu$
and can be assigned to any of the first seven servers.
The jobs of the second type arrive at rate $4 \nu$
and can be assigned to any of the last seven servers.

In [ ]:
# parameters
N = 10
μ = ones(N)
ℓ = nb_tokens * ones(N, dtype = int)

# best static policy
best = asarray([7, 7, 7, 12, 12, 12, 12, 12, 12, 12], dtype = float)
best /= sum(best)

# uniform static policy
uni = asarray([1, 1, 1, 5, 5, 5, 5, 4, 4, 4], dtype = float)
uni /= sum(uni)

In [ ]:
# compute the analytical results
best_static_df = create_static_df(N, ℓ, μ, ρρ, best)
uni_static_df = create_static_df(N, ℓ, μ, ρρ, uni)

In [ ]:
# save the results in csv
best_static_df.to_csv("data/multi-best-static-exact.csv", index = False)
uni_static_df.to_csv("data/multi-uni-static-exact.csv", index = False)